# Magnetic field predictions from axial currents

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
import LFPy
from LFPy import InfiniteHomogeneousVolCondMEG as MEG
from brainsignals.plotting_convention import mark_subplots, simplify_axes, cmap_v_e

In [ ]:

# parameters for Cell instance with simplified morphology
cellParameters = {
    'morphology': 'simplemorpho_modded.hoc',
    'tstop': 10.,      # sim duration
    'dt': 2**-4,       # sim time step size
    'v_init': -65,     # intitial voltage
    'cm': 1.,          # membrane capacitance
    'Ra': 150.,        # axial resistivity
    'passive': True,   # turn on passive mechanism
    'passive_parameters': {'g_pas': 1. / 3E4,   # passive leak conductance
                           'e_pas': -65.},    # leak reversal potential
    'nsegs_method': 'lambda_f',
    'lambda_f': 100.
}
# create cell, set alignment
cell = LFPy.Cell(**cellParameters)
cell.set_rotation(x=np.pi / 2, y=np.pi/32)

# synapse location index
synidx = cell.get_idx(['apic[3]'])[1]

# parameters for synapse instance
synapseParameters = {
    'idx': synidx,
    'e': 0,                                # reversal potential
    'syntype': 'Exp2Syn',                   # synapse type
    'tau1': 0.1,                              # syn. rise time constant
    'tau2': 1.,                              # syn. decay time constant
    'weight': 0.002,                        # syn. weight
    'record_current': True                 # syn. current record
}
# create synapse and set activation time
synapse = LFPy.Synapse(cell, **synapseParameters)
synapse.set_spike_times(np.array([2.]))

# x,y-limits for extracellular predictions
x_lims = [-90., 100.]
y_lims = [-80., 250.]

# y-offset for extracellular predictions
y_offset = -10

# extracellular electrode parameters
dx = 5.  # spatial resolution
x = np.arange(x_lims[0], x_lims[1] + dx, dx)
z = np.arange(y_lims[0], y_lims[1] + dx, dx)
X, Z = np.meshgrid(x, z)
electrodeParameters = {
    'x': X.flatten(),
    'y': np.zeros(X.size) + y_offset,
    'z': Z.flatten(),
    'sigma': 0.3,
    'method': 'linesource',
}
# instantiate electrode
electrode = LFPy.RecExtElectrode(cell=cell, **electrodeParameters)

# compute cell response, current dipole moment and extracellular potential
cell.simulate(probes=[electrode],
              rec_imem=True,
              rec_vmem=True)


# Locations where we compute magnetic fields
dx_p = 10.
x_p = np.arange(x_lims[0], x_lims[1]  + dx_p, dx_p)
z_p = np.arange(y_lims[0], y_lims[1]  + dx_p, dx_p)
X_p, Z_p = np.meshgrid(x_p, z_p)
# X_p[::2, ] += dx_p / 2
Z_p[:, ::2] += dx_p / 2
Y_p = np.zeros(X_p.shape) + y_offset

# Compute magnetic field B assuming infinite homogeneous volume conductor
sensor_locations = np.c_[X_p.ravel(), Y_p.ravel(), Z_p.ravel()]
meg = MEG(sensor_locations)
H = meg.calculate_H_from_iaxial(cell)
B = H * meg.mu

# for plotting, extract axial currents, their displacements and positions
i_axial, d_vectors, pos_vectors = cell.get_axial_currents_from_vmem()

## plot result

In [ ]:
def draw_cbar(fig, ax, im, xoffset=0, label='label'):
    '''draws horizontal colorbar along bottom of axes'''
    bbox = np.array(ax.get_position())
    cax = fig.add_axes([bbox[0][0] + (bbox[1][0] - bbox[0][0]) * xoffset,
                        bbox[0][1], (bbox[1][0] - bbox[0][0]) / 3, 0.015])
    axcb = fig.colorbar(im, cax=cax, orientation='horizontal', extend='both')
    axcb.outline.set_visible(False)
    axcb.set_label(label, labelpad=0)
    return axcb, cax

In [ ]:
# time index which maximize synapse current magnitude 
t_max = np.argmax(np.abs(synapse.i))

fig = plt.figure(figsize=(6, 2.6))
fig.subplots_adjust(left=0.25, wspace=0.3, right=0.92, top=0.96, bottom=0.02)

ax0 = fig.add_subplot(121, aspect=1, frameon=False, xticks=[], yticks=[])

#ax1 = fig.add_subplot(122, aspect=1, sharex=ax0, sharey=ax0, 
#                      frameon=False, xticks=[], yticks=[])  # axial currents

ax2 = fig.add_subplot(122, aspect=1, sharex=ax0, sharey=ax0,
                      frameon=False, xticks=[], yticks=[])  # magnetic field

axes = [ax0, #ax1, 
        ax2]

vmin_vm = -65
vmax_vm = -63
# plot morphology
zips = []
xz = cell.get_idx_polygons()
for x, z in xz:
    zips.append(list(zip(x, z)))
norm = plt.Normalize(vmin=vmin_vm, vmax=vmax_vm, clip=True)
colors = [plt.get_cmap('viridis')(norm(v)) for v in cell.vmem[:, t_max]]

ims = []
for i, ax in enumerate(axes):
    polycol = PolyCollection(zips,
                             linewidths=(0.5),
                             edgecolors=colors if i==0 else '0.9',
                             facecolors=colors if i==0 else '0.9',
                             zorder=-5)
    
    if i == 0:
        im_V_m = ax.add_collection(polycol)
    else:
        ax.add_collection(polycol)

    # mark synapse location
    ax.plot(synapse.x, synapse.z, 'o', c='r', ms=3)
    
# some extracellular position and annotation
ax0.plot(-80, 30, 'o', color='b', ms=3, zorder=0, clip_on=False)
#ax0.text(-80, 40, r'$V_\mathrm{e}({\bf r}, t)$', horizontalalignment='center')

# draw V_e at t==|i_syn|_max
vmax = 1  # color limits
im = ax0.contourf(X, Z, electrode.data[:, t_max].reshape(X.shape) * 1E3,
                  cmap=cmap_v_e, 
                  zorder=-10,
                  norm=plt.Normalize(vmin=-vmax, vmax=vmax, clip=True),
                  levels=51
                 )

m = plt.cm.ScalarMappable(cmap=cmap_v_e)
m.set_clim(-vmax, vmax)

# create axes for synapse input current
synax = fig.add_axes([0.1, 0.75, 0.15, 0.12], ylabel='$i_\mathrm{syn}$ (nA)', 
                     xlabel="time (ms)", title="I", xticks=[0, 5, 10])
synax.plot(cell.tvec, synapse.i, 'r', lw=1)

# axes for somatic voltage
vax = fig.add_axes([0.1, 0.45, 0.15, 0.12], ylabel='$V_\mathrm{m}$ (mV)', 
                     xlabel="time (ms)", title="II", xticks=[0, 5, 10])
vax.plot(cell.tvec, cell.somav, 'r', lw=1)

# create axes for extracellular potential trace
lfpax = fig.add_axes([0.1, 0.15, 0.15, 0.12], ylabel='$V_\mathrm{e}$ (µV)', 
                     xlabel="time (ms)", title="III", xticks=[0, 5, 10])
lfpax.plot(cell.tvec, electrode.data[(
    electrode.x == -80) & (electrode.z == 30), ].ravel() * 1E3, 'b', lw=1)

for ax in [synax, vax, lfpax]:
    ax.axvline(cell.tvec[t_max], ls='--', c='gray')
    simplify_axes(ax)
    
    
ax0.plot([50, 50], [30, 80], 'k', lw=1, clip_on=False)
ax0.text(52, 55, '50\nµm', va='center')
    
# axis cross
ax0.annotate("", xy=(60, -50),
             xycoords='data', xytext=(100, -50), textcoords='data',
             arrowprops=dict(arrowstyle="<|-",
                             connectionstyle="arc3,rad=0", facecolor='black'),
             zorder=0)
ax0.annotate("", xy=(60, -50),
             xycoords='data', xytext=(60, -10), textcoords='data',
             arrowprops=dict(arrowstyle="<|-",
                             connectionstyle="arc3,rad=0", facecolor='black'),
             zorder=0)
ax0.plot(60, -50, 'o', ms=6, mec='k', mfc='w', zorder=1)
ax0.plot(60, -50, 'kx', ms=4, zorder=2)
ax0.text(80, -67, '$x$')
ax0.text(55, -70, '$y$')
ax0.text(65, -25, '$z$')

# axial currents times displacement
#im3 = ax1.quiver(pos_vectors[:, 0], 
#                 pos_vectors[:, 2], 
#                 d_vectors[0, :]*np.sign(i_axial[:, t_max].ravel()), 
#                 d_vectors[2, :]*np.sign(i_axial[:, t_max].ravel()),
#                 abs(i_axial[:, t_max].ravel())*1E3, 
#                 angles='xy', scale_units='xy', scale=1,
#                 width=8E-3 * 2,
#                 zorder=1,
#                 pivot='mid',
#                 cmap=plt.get_cmap('magma'),
#                clim=[0, 80])


# Plot scalar x,z-component of magnetic field B at t=|i_syn|_max
im4 = ax2.quiver(X_p, Z_p,
                # mT -> fT unit conversion
                B[:, 0, t_max].reshape(X_p.shape) * 1E12,
                B[:, 2, t_max].reshape(X_p.shape) * 1E12,
                B[:, 1, t_max].reshape(X_p.shape) * 1E12,
                cmap=plt.get_cmap('plasma'), 
                width=8E-3,
                zorder=10)

cax1 = fig.add_axes([0.52, 0.17, 0.007, 0.3], frameon=False)
cbar1 = plt.colorbar(m, cax=cax1)
cbar1.set_label(r'$V_\mathrm{e}$ (µV)', labelpad=2)

cax2 = fig.add_axes([0.52, 0.545, 0.007, 0.3], frameon=False)
cbar2 = plt.colorbar(im_V_m, cax=cax2,)
cbar2.set_label(r'$V_\mathrm{m}$ (mV)', labelpad=2)
cbar2.set_ticks([0, 0.5, 1])
cbar2.set_ticklabels([vmin_vm, int((vmax_vm + vmin_vm)/2), vmax_vm])

#cax3 = fig.add_axes([0.62, 0.545, 0.007, 0.3], frameon=False)
#cbar3 = plt.colorbar(im3, cax=cax3, ticks=[0, 25, 50, 75])
#cbar3.set_label(r'$I_{\rm a}$ (pA)', labelpad=2)

cax4 = fig.add_axes([0.915, 0.545, 0.007, 0.3], frameon=False)
cbar4 = plt.colorbar(im4, cax=cax4)
cbar4.set_label('$B_y$ (fT)', labelpad=-5)

mark_subplots(ax0, "A", ypos=1.0, xpos=-0.95)
#mark_subplots(ax1, "B", ypos=1.0, xpos=0.15)
mark_subplots(ax2, "B", ypos=1.0, xpos=0.05)

fig.savefig('Fig-5_redrawn_2.pdf')